In [0]:
%python
dbutils.secrets.list('straidlearner')

Out[1]: [SecretMetadata(key='onlyread'),
 SecretMetadata(key='readstorage'),
 SecretMetadata(key='secretclient'),
 SecretMetadata(key='storageaccount'),
 SecretMetadata(key='storageread'),
 SecretMetadata(key='storagewrite')]

In [0]:
service_credential = dbutils.secrets.get(scope="straidlearner",key="secretclient")

spark.conf.set("fs.azure.account.auth.type.mijandatalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.mijandatalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.mijandatalake.dfs.core.windows.net", "c8457ca3-4238-4db0-8499-2f727e79bd6f")
spark.conf.set("fs.azure.account.oauth2.client.secret.mijandatalake.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.mijandatalake.dfs.core.windows.net", "https://login.microsoftonline.com/264702b4-b846-47db-85d5-489e1ebdf13f/oauth2/token")

In [0]:
%fs ls dbfs:/mnt

path,name,size,modificationTime
dbfs:/mnt/straidcontainer/,straidcontainer/,0,0
dbfs:/mnt/training/,training/,0,0


In [0]:
# configs = {
#   "fs.azure.account.auth.type": "CustomAccessToken",
#   "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
# }

# Optionally, you can add <directory-name> to the source URI of your mount point.
# dbutils.fs.mount(
#   source = "wasbs://bronze@mijandatalake.blob.core.windows.net/",
#   mount_point = "/mnt/bronze",
#   extra_configs = {"fs.azure.account.key.mijandatalake.blob.core.windows.net":"OEGS0kzsdL05loPlph72vAKGbHjrgcpLsdHVn9WDkKFF0ecLfq6+dKO0riMf9yarllyW+b1gAeFv+AStn96lqA=="})

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1559368678780362>:7
      1 # configs = {
      2 #   "fs.azure.account.auth.type": "CustomAccessToken",
      3 #   "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
      4 # }
      5 
      6 # Optionally, you can add <directory-name> to the source URI of your mount point.
----> 7 dbutils.fs.mount(
      8   source = "abfss://bronze@mijandatalake.dfs.core.windows.net/",
      9   mount_point = "/mnt/bronze",
     10   extra_configs = {"fs.azure.account.key.mijandatalake.blob.core.windows.net":"OEGS0kzsdL05loPlph72vAKGbHjrgcpLsdHVn9WDkKFF0ecLfq6+dKO0riMf9yarllyW+b1gAeFv+AStn96lqA=="})

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    

In [0]:
storageAccountName = "mijandatalake"
storageAccountAccessKey = "OEGS0kzsdL05loPlph72vAKGbHjrgcpLsdHVn9WDkKFF0ecLfq6+dKO0riMf9yarllyW+b1gAeFv+AStn96lqA=="
#sasToken = <sas-token>
blobContainerName = "bronze"
mountPoint = "/mnt/bronze/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
      #extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

mount succeeded!


In [0]:
dbutils.fs.mounts()

Out[11]: [MountInfo(mountPoint='/mnt/training', source='wasbs://training@dbtraineastus.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/bronze/', source='wasbs://bronze@mijandatalake.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/mnt/straidcontainer', source='wasbs://straidcontainer@straid.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsRes

In [0]:
# %python

# MOUNTPOINT = "/mnt/bronze"

# if MOUNTPOINT in [mnt.mountPoint for mnt in dbutils.fs.mounts()]:
#   dbutils.fs.unmount(MOUNTPOINT)

/mnt/bronze has been unmounted.


In [0]:
dbutils.fs.ls("/mnt/bronze")

Out[12]: [FileInfo(path='dbfs:/mnt/bronze/etl/', name='etl/', size=0, modificationTime=0)]

In [0]:

%fs ls dbfs:/mnt/bronze/etl/car_price

path,name,size,modificationTime
dbfs:/mnt/bronze/etl/car_price/car_price.parquet,car_price.parquet,197984,1692740467000


In [0]:
#CREATE A MOUNT POINT FOR SILVER CONTAINER

storageAccountName = "mijandatalake"
storageAccountAccessKey = "OEGS0kzsdL05loPlph72vAKGbHjrgcpLsdHVn9WDkKFF0ecLfq6+dKO0riMf9yarllyW+b1gAeFv+AStn96lqA=="
#sasToken = <sas-token>
blobContainerName = "silver"
mountPoint = "/mnt/silver/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
      #extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

mount succeeded!


In [0]:
#CREATE A MOUNT POINT FOR GOLD CONTAINER

storageAccountName = "mijandatalake"
storageAccountAccessKey = "OEGS0kzsdL05loPlph72vAKGbHjrgcpLsdHVn9WDkKFF0ecLfq6+dKO0riMf9yarllyW+b1gAeFv+AStn96lqA=="
#sasToken = <sas-token>
blobContainerName = "gold"
mountPoint = "/mnt/gold/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
      #extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

mount succeeded!


In [0]:
dbutils.fs.mounts()

Out[16]: [MountInfo(mountPoint='/mnt/training', source='wasbs://training@dbtraineastus.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/mnt/gold/', source='wasbs://gold@mijandatalake.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/mnt/silver/', source='wasbs://silver@mijandatalake.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/bronze/', source='wasbs://bronze@mijandatalake.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint